In [79]:
import numpy as np
from tensorflow.keras.models import load_model
import lightgbm as lgb
import pandas as pd
import tensorflow as tf
from sklearn.metrics import confusion_matrix
import os
import cv2
import xgboost as xgb
#import pandas_ml as pdml
import os
import pandas as pd


In [80]:

clinical_data = pd.read_csv('clinical_best.csv')
image_directory = 'PREPROCESS_IMAGES/alzheimer_new_2/Axial'
ptids = []
class_labels = []
image_files = []

for class_label in os.listdir(image_directory):
    class_label_path = os.path.join(image_directory, class_label)
    if os.path.isdir(class_label_path):
        for file_name in os.listdir(class_label_path):
            ptid = file_name[2:12] 
            #print(ptid)
            ptids.append(ptid)
            class_labels.append(class_label)
            image_file = os.path.join(class_label_path, file_name)
            image_files.append(image_file)
            
image_data = pd.DataFrame({'PTID': ptids, 'Class_Label': class_labels, 'Image_File': image_files})

merged_data = pd.merge(clinical_data, image_data, on='PTID', how='inner')

merged_data = merged_data.groupby('PTID', as_index=False).agg({'Class_Label': 'first', 'Image_File': list})
merged_data = pd.merge(merged_data, clinical_data, on='PTID', how='left')


filtered_image_data = image_data[image_data['PTID'].isin(merged_data['PTID'])]


print(merged_data)


          PTID Class_Label                                         Image_File  \
0   002_S_0413          CN  [PREPROCESS_IMAGES/alzheimer_new_2/Axial\CN\CN...   
1   002_S_0559          CN  [PREPROCESS_IMAGES/alzheimer_new_2/Axial\CN\CN...   
2   002_S_0816          AD  [PREPROCESS_IMAGES/alzheimer_new_2/Axial\AD\AD...   
3   002_S_1018          AD  [PREPROCESS_IMAGES/alzheimer_new_2/Axial\AD\AD...   
4   002_S_1280          CN  [PREPROCESS_IMAGES/alzheimer_new_2/Axial\CN\CN...   
..         ...         ...                                                ...   
91  136_S_0300          AD  [PREPROCESS_IMAGES/alzheimer_new_2/Axial\AD\AD...   
92  136_S_0426          AD  [PREPROCESS_IMAGES/alzheimer_new_2/Axial\AD\AD...   
93  136_S_0429          CI  [PREPROCESS_IMAGES/alzheimer_new_2/Axial\CI\CI...   
94  136_S_0579          CI  [PREPROCESS_IMAGES/alzheimer_new_2/Axial\CI\CI...   
95  136_S_1227          CI  [PREPROCESS_IMAGES/alzheimer_new_2/Axial\CI\CI...   

    Unnamed: 0  index   RID

In [81]:
class_counts=merged_data["Class_Label"].value_counts()
print(class_counts)

Class_Label
CN    34
AD    33
CI    29
Name: count, dtype: int64


IGNORE 3 CODE BLOCKS--NOT BEING USED

Found 5154 files belonging to 3 classes.
Images shape: (5154, 128, 128, 1)
Labels shape: (5154,)


In [8]:
clinical_data=pd.read_csv('augmented_clinical_data.csv').values

In [27]:
root_folder = "PREPROCESS_IMAGES/alzheimer_new_2/Axial"  
image_data, labels1 = load_images_with_labels(root_folder)
labels = clinical_data[:,3].astype(int)
labels

Found 5154 files belonging to 3 classes.


array([1, 0, 1, ..., 1, 1, 0])

CONTINUE--MODEL BUILDING AND MAKING PREDICTIONS-- try to use feature importance in case of xgb model

In [82]:
clinical_data

,Unnamed: 0,index,PTID,RID,Phase,Group,PTGENDER_-4.0,PTGENDER_1.0,PTGENDER_2.0,PTWORK_-4.0,...,PHC_VSP,PHC_EXF,PHC_MEM,PTDOB,PHC_LAN,VISDATE,PTDOBYY,PTADBEG,PTCOGBEG,PTADDX
0,0,0,002_S_0295,295,ADNI1,0.0,False,True,False,True,...,0.483,0.554,1.357,06/1921,0.272,2006-04-04,1921.0,-4.0,NaN,NaN
1,1,1,002_S_0413,413,ADNI1,0.0,False,False,True,True,...,NaN,1.480,1.287,12/1929,2.594,2006-04-06,1929.0,-4.0,NaN,NaN
2,2,2,002_S_0559,559,ADNI1,0.0,False,True,False,True,...,NaN,1.176,0.768,01/1927,1.041,2006-05-11,1927.0,-4.0,NaN,NaN
3,3,3,002_S_0619,619,ADNI1,2.0,False,True,False,True,...,0.264,-0.509,-1.259,12/1928,0.116,2006-05-18,1928.0,1993.0,NaN,NaN
4,4,4,002_S_0685,685,ADNI1,0.0,False,False,True,True,...,NaN,0.111,0.149,11/1916,1.170,2006-06-22,1916.0,-4.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2227,2227,2227,941_S_6580,6580,ADNI3,0.0,False,False,True,True,...,-0.583,-0.208,0.082,10/1938,0.159,2018-08-31,1938.0,NaN,9999.0,9999.0
2228,2228,2228,941_S_6581,6581,ADNI3,0.0,False,False,True,True,...,NaN,0.824,1.142,03/1944,1.281,2018-09-05,1944.0,NaN,9999.0,9999.0
2229,2229,2229,941_S_6607,6607,ADNI3,0.0,False,True,False,True,...,NaN,NaN,0.624,01/1936,NaN,2018-10-15,1936.0,NaN,9999.0,9999.0
2230,2230,2230,941_S_6803,6803,ADNI3,1.0,False,False,True,True,...,-0.041,-0.854,-0.604,08/1944,-0.147,2019-08-27,1944.0,NaN,2014.0,9999.0


In [67]:
clinical_data.dropna(inplace=True)

In [83]:
import cv2
cnn_model = load_model('clf_model_final.h5')

lgbm_model = lgb.Booster(model_file='lgbm_model.txt')

xgbm_model = xgb.Booster()
xgbm_model.load_model('xgbm_model.model')
preprocessed_images = []

import cv2
import numpy as np

preprocessed_images = []

for index, row in merged_data.iterrows():
    if isinstance(row['Image_File'], list) and any(pd.notna(image_path) for image_path in row['Image_File']):
        for image_path in row['Image_File']:
            image = cv2.imread(str(image_path), cv2.IMREAD_GRAYSCALE)
            image_resized = cv2.resize(image, (128, 128))
            image_rgb = cv2.cvtColor(image_resized, cv2.COLOR_GRAY2RGB)
            image_normalized = image_rgb / 255.0
            preprocessed_images.append(image_normalized)

preprocessed_images = np.array(preprocessed_images)

print(preprocessed_images.shape)

clinical_data = merged_data.drop(columns=['PTID', 'Image_File','PTDOB','Phase','VISDATE','Class_Label','Group'])

#clinical_data.dropna(axis=1, inplace=True)

cnn_pred = cnn_model.predict(preprocessed_images)

lgbm_preds = lgbm_model.predict(clinical_data)

clinical_data_xg=xgb.DMatrix(clinical_data)

xgbm_pred=xgbm_model.predict(clinical_data_xg)


(3232, 128, 128, 3)
101/101 ━━━━━━━━━━━━━━━━━━━━ 18s 176ms/step


In [104]:
lgbm_preds

array([[0.65603643, 0.25783164, 0.08613193],
       [0.65603643, 0.25783164, 0.08613193],
       [0.28824027, 0.3093672 , 0.40239253],
       [0.63280625, 0.17069576, 0.196498  ],
       [0.65603643, 0.25783164, 0.08613193],
       [0.61658172, 0.29587635, 0.08754193],
       [0.647595  , 0.26415924, 0.08824575],
       [0.15042687, 0.18089003, 0.6686831 ],
       [0.23066388, 0.63719993, 0.13213619],
       [0.13466625, 0.16382578, 0.70150798],
       [0.13507996, 0.12123512, 0.74368493],
       [0.27745143, 0.57497975, 0.14756882],
       [0.63085773, 0.28631608, 0.08282619],
       [0.194047  , 0.66681083, 0.13914217],
       [0.60775095, 0.30269088, 0.08955817],
       [0.13854013, 0.19695548, 0.66450439],
       [0.19603561, 0.66588067, 0.13808372],
       [0.19691819, 0.67667718, 0.12640463],
       [0.13507996, 0.12123512, 0.74368493],
       [0.13507996, 0.12123512, 0.74368493],
       [0.63085773, 0.28631608, 0.08282619],
       [0.65603643, 0.25783164, 0.08613193],
       [0.

In [105]:
cnn_pred

<tf.Tensor: shape=(3232, 3), dtype=float32, numpy=
array([[0.575742  , 0.21214055, 0.2121175 ],
       [0.57610184, 0.2119458 , 0.21195237],
       [0.57606447, 0.21195386, 0.21198173],
       ...,
       [0.21194386, 0.5761136 , 0.21194252],
       [0.21194835, 0.57610637, 0.21194525],
       [0.21196704, 0.5760251 , 0.21200784]], dtype=float32)>

In [85]:
pd.DataFrame(cnn_pred).to_csv("uniquecnn.csv")
pd.DataFrame(lgbm_preds).to_csv("uniquelgbm.csv")
pd.DataFrame(xgbm_pred).to_csv("uniquexgbm.csv")


In [74]:
len(cnn_pred)

3232

In [75]:
len(filtered_image_data)

3232

To get one label for all of the images of a single patient

In [86]:
aggregated_predictions = {}
for i, ptid in enumerate(filtered_image_data['PTID']):
    if ptid not in aggregated_predictions:
        aggregated_predictions[ptid] = cnn_pred[i]
    else:
        aggregated_predictions[ptid] += cnn_pred[i]

for ptid in aggregated_predictions:
    aggregated_predictions[ptid] /= len(image_data[image_data['PTID'] == ptid])

In [87]:
aggregated_predictions

{'002_S_0816': array([9.9835384e-01, 3.5824307e-04, 1.2877497e-03], dtype=float32),
 '002_S_1018': array([0.98437506, 0.00851073, 0.00711426], dtype=float32),
 '005_S_0814': array([2.4978116e-01, 1.6678240e-04, 7.5005203e-01], dtype=float32),
 '007_S_1304': array([1.7006158e-05, 5.1618499e-06, 9.9997789e-01], dtype=float32),
 '012_S_0689': array([1.3897391e-05, 6.4353918e-07, 9.9998564e-01], dtype=float32),
 '018_S_0335': array([6.623981e-05, 5.743341e-09, 9.999336e-01], dtype=float32),
 '018_S_0633': array([0.7881152 , 0.00171335, 0.2101715 ], dtype=float32),
 '021_S_0753': array([0.48656535, 0.5091039 , 0.00433073], dtype=float32),
 '023_S_0139': array([0.503343  , 0.00201997, 0.49463686], dtype=float32),
 '023_S_0916': array([2.8880173e-01, 7.0235244e-04, 7.1049589e-01], dtype=float32),
 '023_S_1262': array([0.62222326, 0.34453088, 0.03324582], dtype=float32),
 '023_S_1289': array([0.00462274, 0.66813594, 0.32724145], dtype=float32),
 '027_S_0404': array([3.6796305e-01, 3.7202108e-0

In [78]:
len(aggregated_predictions)

96

In [88]:
patients_predictions = []
for ptid in aggregated_predictions:
    patients_predictions.append(aggregated_predictions[ptid])

patients_predictions_array = np.array(patients_predictions)

In [33]:
len(patients_predictions_array)

96

In [212]:
clinical_data

,Unnamed: 0,index,RID,PTGENDER_-4.0,PTGENDER_1.0,PTGENDER_2.0,PTWORK_-4.0,PTHOME_-4.0,PTHOME_1.0,PTHOME_2.0,...,NXTENDON_2.0,AGE,PHC_VSP,PHC_EXF,PHC_MEM,PHC_LAN,PTDOBYY,PTADBEG,PTCOGBEG,PTADDX
0,1,1,413,False,False,True,True,False,True,False,...,False,76.9528,NaN,1.48,1.287,2.594,1929.0,-4.0,NaN,NaN
1,1,1,413,False,False,True,True,False,True,False,...,False,76.9528,NaN,1.48,1.287,2.594,1929.0,-4.0,NaN,NaN
2,1,1,413,False,False,True,True,False,True,False,...,False,76.9528,NaN,1.48,1.287,2.594,1929.0,-4.0,NaN,NaN
3,1,1,413,False,False,True,True,False,True,False,...,False,76.9528,NaN,1.48,1.287,2.594,1929.0,-4.0,NaN,NaN
4,1,1,413,False,False,True,True,False,True,False,...,False,76.9528,NaN,1.48,1.287,2.594,1929.0,-4.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3227,1953,1953,1227,False,False,True,True,False,True,False,...,True,65.2375,-0.237,0.61,0.284,0.727,1942.0,-4.0,NaN,NaN
3228,1953,1953,1227,False,False,True,True,False,True,False,...,True,65.2375,-0.237,0.61,0.284,0.727,1942.0,-4.0,NaN,NaN
3229,1953,1953,1227,False,False,True,True,False,True,False,...,True,65.2375,-0.237,0.61,0.284,0.727,1942.0,-4.0,NaN,NaN
3230,1953,1953,1227,False,False,True,True,False,True,False,...,True,65.2375,-0.237,0.61,0.284,0.727,1942.0,-4.0,NaN,NaN


In [207]:
clinical_data.shape

(3232, 118)

In [ ]:
label_counts = pd.Series(y).value_counts()
print(label_counts)

1    2590
0    1440
2    1124
Name: count, dtype: int64


In [7]:

cnn_pred_save=pd.DataFrame(cnn_pred).to_csv("cnn_pred.csv")
lgbm_pred_save=pd.DataFrame(lgbm_preds).to_csv("lgbm_pred.csv")
xgbm_pred_save=pd.DataFrame(xgbm_pred).to_csv("xgbm_pred.csv")

In [167]:
preprocessed_images.shape

(3232, 128, 128, 3)

Just executing this once, else would have been done above--not needed --just for seeing shape

In [119]:
cnn_pred_labels = np.array([np.argmax(y_) for y_ in cnn_pred])
cnn_pred_labels


array([2, 2, 2, ..., 1, 1, 1], dtype=int64)

In [80]:
clinical_data.shape


(3232, 118)

IGNORE-- the predictions have already been made

In [89]:
class_label_mapping = {'AD': 2, 'MCI': 1, 'CN': 0}
merged_data['Class_Label'] = merged_data['Class_Label'].map(class_label_mapping)

In [90]:
clinical_data = merged_data.drop(columns=['PTID', 'Image_File','PTDOB','Phase','VISDATE','Class_Label','Group'])


cnn_pred = cnn_model.predict(preprocessed_images)

lgbm_preds = lgbm_model.predict(clinical_data)


101/101 ━━━━━━━━━━━━━━━━━━━━ 17s 164ms/step


ANALYSING THE PREDICTIONS

In [13]:
lgbm_preds

array([[0.65603643, 0.25783164, 0.08613193],
       [0.65603643, 0.25783164, 0.08613193],
       [0.28824027, 0.3093672 , 0.40239253],
       [0.63280625, 0.17069576, 0.196498  ],
       [0.65603643, 0.25783164, 0.08613193],
       [0.61658172, 0.29587635, 0.08754193],
       [0.647595  , 0.26415924, 0.08824575],
       [0.15042687, 0.18089003, 0.6686831 ],
       [0.23066388, 0.63719993, 0.13213619],
       [0.13466625, 0.16382578, 0.70150798],
       [0.13507996, 0.12123512, 0.74368493],
       [0.27745143, 0.57497975, 0.14756882],
       [0.63085773, 0.28631608, 0.08282619],
       [0.194047  , 0.66681083, 0.13914217],
       [0.60775095, 0.30269088, 0.08955817],
       [0.13854013, 0.19695548, 0.66450439],
       [0.19603561, 0.66588067, 0.13808372],
       [0.19691819, 0.67667718, 0.12640463],
       [0.13507996, 0.12123512, 0.74368493],
       [0.13507996, 0.12123512, 0.74368493],
       [0.63085773, 0.28631608, 0.08282619],
       [0.65603643, 0.25783164, 0.08613193],
       [0.

In [238]:
cnn_pred

array([[9.9897671e-01, 5.6596845e-04, 4.5732732e-04],
       [9.9995899e-01, 4.9829773e-06, 3.5999510e-05],
       [9.9985683e-01, 5.8040455e-06, 1.3728770e-04],
       ...,
       [7.6184165e-06, 9.9999106e-01, 1.2837530e-06],
       [2.1637859e-05, 9.9997139e-01, 6.9874350e-06],
       [2.8938684e-05, 9.9974960e-01, 2.2143488e-04]], dtype=float32)

In [12]:
xgbm_pred

array([[0.56322557, 0.27638593, 0.1603885 ],
       [0.5540834 , 0.28376406, 0.1621525 ],
       [0.22479747, 0.24248566, 0.53271693],
       [0.36006615, 0.19233516, 0.44759867],
       [0.55344045, 0.2840479 , 0.16251168],
       [0.37411922, 0.4373486 , 0.18853216],
       [0.55153376, 0.2853866 , 0.16307966],
       [0.16737331, 0.18128413, 0.6513425 ],
       [0.31475365, 0.48820654, 0.19703975],
       [0.14968011, 0.16336462, 0.6869553 ],
       [0.14739823, 0.15966061, 0.6929411 ],
       [0.30702507, 0.49546552, 0.19750935],
       [0.53189284, 0.30670777, 0.16139941],
       [0.23716809, 0.5433277 , 0.21950427],
       [0.40332907, 0.4065131 , 0.19015785],
       [0.15337054, 0.25017908, 0.5964504 ],
       [0.56322557, 0.27638593, 0.1603885 ],
       [0.22596525, 0.53856224, 0.23547255],
       [0.14566441, 0.17316446, 0.6811712 ],
       [0.1568511 , 0.17094037, 0.67220855],
       [0.53744775, 0.29895043, 0.1636018 ],
       [0.5153605 , 0.31451476, 0.17012477],
       [0.

USE PREDICTIONS TO GET WEIGHTED AVERAGE
ALTER AS NEEDED

In [98]:

cnn_weight = 0.4
lgbm_weight =0.6
#xgbm_weight=0.6

In [36]:
patients_predictions_array

<tf.Tensor: shape=(96, 3), dtype=float32, numpy=
array([[0.78329825, 0.1082501 , 0.10845172],
       [0.7751934 , 0.11256459, 0.11224208],
       [0.22484446, 0.1414296 , 0.63372594],
       [0.11076509, 0.11076247, 0.77847236],
       [0.11037184, 0.11036891, 0.7792593 ],
       [0.10881405, 0.10879964, 0.7823863 ],
       [0.6639645 , 0.1377483 , 0.19828711],
       [0.40507913, 0.43603632, 0.15888461],
       [0.42256218, 0.15694493, 0.4204929 ],
       [0.2605959 , 0.14646044, 0.59294367],
       [0.52725273, 0.30773103, 0.16501626],
       [0.15075448, 0.5677313 , 0.28151423],
       [0.3194947 , 0.15317135, 0.52733403],
       [0.7807624 , 0.10966261, 0.10957496],
       [0.77702224, 0.11214648, 0.11083119],
       [0.19847302, 0.13661286, 0.66491413],
       [0.17212303, 0.12952833, 0.6983486 ],
       [0.7759861 , 0.11213915, 0.11187476],
       [0.77411234, 0.1137557 , 0.1121319 ],
       [0.7769308 , 0.1119205 , 0.11114869],
       [0.7757853 , 0.11280932, 0.11140537],
      

In [9]:
cnn_pred

array([[9.9897671e-01, 5.6596845e-04, 4.5732732e-04],
       [9.9995899e-01, 4.9829773e-06, 3.5999510e-05],
       [9.9985683e-01, 5.8040455e-06, 1.3728770e-04],
       ...,
       [7.6184165e-06, 9.9999106e-01, 1.2837530e-06],
       [2.1637859e-05, 9.9997139e-01, 6.9874350e-06],
       [2.8938684e-05, 9.9974960e-01, 2.2143488e-04]], dtype=float32)

In [91]:
cnn_pred=tf.nn.softmax(cnn_pred)

In [92]:
patients_predictions_array=tf.nn.softmax(patients_predictions_array)

In [54]:
patients_predictions_array

<tf.Tensor: shape=(96, 3), dtype=float32, numpy=
array([[0.78329825, 0.1082501 , 0.10845172],
       [0.7751934 , 0.11256459, 0.11224208],
       [0.22484446, 0.1414296 , 0.63372594],
       [0.11076509, 0.11076247, 0.77847236],
       [0.11037184, 0.11036891, 0.7792593 ],
       [0.10881405, 0.10879964, 0.7823863 ],
       [0.6639645 , 0.1377483 , 0.19828711],
       [0.40507913, 0.43603632, 0.15888461],
       [0.42256218, 0.15694493, 0.4204929 ],
       [0.2605959 , 0.14646044, 0.59294367],
       [0.52725273, 0.30773103, 0.16501626],
       [0.15075448, 0.5677313 , 0.28151423],
       [0.3194947 , 0.15317135, 0.52733403],
       [0.7807624 , 0.10966261, 0.10957496],
       [0.77702224, 0.11214648, 0.11083119],
       [0.19847302, 0.13661286, 0.66491413],
       [0.17212303, 0.12952833, 0.6983486 ],
       [0.7759861 , 0.11213915, 0.11187476],
       [0.77411234, 0.1137557 , 0.1121319 ],
       [0.7769308 , 0.1119205 , 0.11114869],
       [0.7757853 , 0.11280932, 0.11140537],
      

In [94]:
cnn_pred1=pd.DataFrame(cnn_pred)
cnn_pred1.to_csv("uniquecnn.csv")
xgbm_preds1=pd.DataFrame(patients_predictions_array)
xgbm_preds1.to_csv("uniquexgb.csv")

In [11]:
cnn_pred

<tf.Tensor: shape=(3232, 3), dtype=float32, numpy=
array([[0.575742  , 0.21214055, 0.2121175 ],
       [0.57610184, 0.2119458 , 0.21195237],
       [0.57606447, 0.21195386, 0.21198173],
       ...,
       [0.21194386, 0.5761136 , 0.21194252],
       [0.21194835, 0.57610637, 0.21194525],
       [0.21196704, 0.5760251 , 0.21200784]], dtype=float32)>

In [242]:
cnn_pred1=pd.DataFrame(cnn_pred)
cnn_pred1.to_csv("cnn.csv")

: 

In [152]:
lgbm_preds1=pd.DataFrame(lgbm_preds)
lgbm_preds1.to_csv("lgbm.csv")

Two blocks above were to save the predictions in case needed to cross-verify

TensorShape([3232, 3])

In [97]:
weighted_preds = (cnn_weight* patients_predictions_array) + (lgbm_weight * lgbm_preds)
#weighted_preds = (cnn_weight* cnn_pred) + (xgbm_weight * xgbm_pred)
#weighted_preds=xgbm_pred


In [45]:
print(weighted_preds)


tf.Tensor(
[[0.694215   0.21295717 0.09282787]
 [0.69178355 0.21425152 0.09396498]
 [0.26922154 0.25898594 0.47179258]
 [0.4761939  0.15271577 0.3710903 ]
 [0.49233705 0.21359283 0.29407015]
 [0.4642514  0.23975334 0.29599524]
 [0.6525059  0.22623597 0.12125816]
 [0.22682256 0.25743392 0.51574355]
 [0.2882334  0.4931234  0.21864319]
 [0.17244515 0.15861617 0.6689387 ]
 [0.2527318  0.1771839  0.57008433]
 [0.23944235 0.5728052  0.18775244]
 [0.5374488  0.24637267 0.21617855]
 [0.37006164 0.49966636 0.130272  ]
 [0.6585324  0.24552757 0.09594008]
 [0.15652    0.17885269 0.6646273 ]
 [0.18886185 0.50497496 0.3061632 ]
 [0.37063855 0.50731575 0.12204567]
 [0.32678968 0.11899129 0.554219  ]
 [0.32763523 0.11844073 0.5539241 ]
 [0.674336   0.23426405 0.09139995]
 [0.6874296  0.21526265 0.09730777]
 [0.19685876 0.26577386 0.5373674 ]
 [0.17097199 0.50671756 0.32231045]
 [0.12785578 0.1181706  0.7539736 ]
 [0.28543323 0.5158012  0.19876558]
 [0.47754702 0.37655318 0.14589979]
 [0.17142871 0.50

In [110]:
print(merged_data)

      Unnamed: 0  index        PTID   RID  Phase  Group  PTGENDER_-4.0  \
0              1      1  002_S_0413   413  ADNI1    0.0          False   
1              1      1  002_S_0413   413  ADNI1    0.0          False   
2              1      1  002_S_0413   413  ADNI1    0.0          False   
3              1      1  002_S_0413   413  ADNI1    0.0          False   
4              1      1  002_S_0413   413  ADNI1    0.0          False   
...          ...    ...         ...   ...    ...    ...            ...   
3227        1953   1953  136_S_1227  1227  ADNI1    1.0          False   
3228        1953   1953  136_S_1227  1227  ADNI1    1.0          False   
3229        1953   1953  136_S_1227  1227  ADNI1    1.0          False   
3230        1953   1953  136_S_1227  1227  ADNI1    1.0          False   
3231        1953   1953  136_S_1227  1227  ADNI1    1.0          False   

      PTGENDER_1.0  PTGENDER_2.0  PTWORK_-4.0  ...  PHC_MEM    PTDOB  PHC_LAN  \
0            False          Tr

NO NEED--ONYL FOR ANALYSIS

In [64]:
merged_data.to_csv("merged.csv")

In [57]:
merged_data.to_csv("merged_new.csv")

In [41]:
merged_data['Group'] = merged_data.groupby('Class_Label')['Group'].transform('first')


In [127]:
weighted_preds.shape

TensorShape([3232, 3])

HERE

In [99]:
y_pred = np.argmax(weighted_preds, axis=1)
y_pred


array([0, 0, 2, 0, 0, 0, 0, 2, 1, 2, 2, 1, 0, 1, 0, 2, 1, 1, 2, 2, 0, 0,
       2, 1, 2, 1, 0, 1, 2, 2, 0, 0, 2, 1, 0, 2, 1, 2, 2, 0, 0, 2, 0, 1,
       1, 1, 0, 0, 1, 1, 1, 2, 2, 2, 2, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 2, 2, 2, 0, 1, 2, 0, 0, 1, 1, 2, 1, 0, 0, 0, 0, 2, 0, 2, 0,
       2, 1, 2, 0, 2, 1, 0, 0], dtype=int64)

In [100]:
y_true=merged_data['Group']
y_true

0     0.0
1     0.0
2     2.0
3     2.0
4     0.0
     ... 
91    2.0
92    2.0
93    1.0
94    1.0
95    1.0
Name: Group, Length: 96, dtype: float64

In [65]:
#THESE WERE CLASS COUNTS

Class_Label
AD    1124
CI    1074
CN    1034
Name: count, dtype: int64


EVALUATION METRICS

In [52]:
conf_matrix = confusion_matrix(y_true, y_pred)

num_classes = conf_matrix.shape[0]
recall_per_class = [conf_matrix[i, i] / np.sum(conf_matrix[i, :]) for i in range(num_classes)]

avg_recall = np.mean(recall_per_class)

for i in range(num_classes):
    print(f'Recall for class {i}: {recall_per_class[i]}')


Recall for class 0: 0.7647058823529411
Recall for class 1: 0.6551724137931034
Recall for class 2: 0.8787878787878788


In [103]:
from sklearn.metrics import classification_report

report = classification_report(y_true, y_pred, target_names=['class_0:CN', 'class_1:MCI','class 2:AD'])

print(report)


              precision    recall  f1-score   support

  class_0:CN       0.66      0.79      0.72        34
 class_1:MCI       0.72      0.62      0.67        29
  class 2:AD       1.00      0.91      0.95        33

    accuracy                           0.78        96
   macro avg       0.79      0.77      0.78        96
weighted avg       0.79      0.78      0.78        96

